# Determinantes dos preços dos restaurantes de SP disponíveis no TripAdvisor<br>Fase 1: Extração

## 0. Pré-Scrapping
***
### Importação dos pacotes e definição de funções:

In [1]:
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException, ElementClickInterceptedException, TimeoutException

import time
from time import sleep

from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
# determinando parâmetro options para a função que define o webdriver:
options = Options()
# para o browser não abrir durante o processo:
# options.add_argument("--headless")

In [3]:
# criar e atualizar variáveis html e pagina que transformam página atual do webdriver em um arquivo soup:
def atualiza_transforma_pagina(webdriver):
    global html, pagina # transforma variáveis html e pagina em globais, ou seja, não ficam restritas a função.
    html = browser.page_source
    pagina = BeautifulSoup(html, 'html.parser')
    
# clicar no botão que aceita os cookies e desobstruir página para realização da raspagem:     
def aceita_cookies(webdriver, id_):
    time.sleep(7)
    botao_aceita_cookies = webdriver.find_element(By.ID, str(id_))
    botao_aceita_cookies.click()

# clicar no botão de próxima página
def proxima_pagina(webdriver, css_locator):
    
    time.sleep(3)
    wait = WebDriverWait(webdriver, 10, ignored_exceptions=[StaleElementReferenceException, WebDriverException])
    global fim_da_coleta
    fim_da_coleta = 0
        
    try:
        proxima_pagina = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, str(css_locator))))
        proxima_pagina.click()
        time.sleep(3)
        
    except ElementClickInterceptedException:
        fim_da_coleta = 1
        print('Coleta finalizada!')
        
    except TimeoutException:
        fim_da_coleta = 1
        print('Erro "TimeoutException": coleta pode ter sido finalizada. Verificar!')

## 1. Scrapping
***

In [18]:
# abrindo uma lista em branco para incluir dicionários com nomes dos restaurantes e url das páginas:
cards = []

browser = webdriver.Firefox(options=options, executable_path="C:\Program Files\Mozilla Firefox\geckodriver.exe")
url = 'https://www.tripadvisor.com.br/Restaurants-g303631-Sao_Paulo_State_of_Sao_Paulo.html'
browser.get(url)

atualiza_transforma_pagina(browser)
aceita_cookies(browser, 'onetrust-accept-btn-handler')

while True:
    atualiza_transforma_pagina(browser)

    restaurantes = pagina.find_all('div', {'data-test': re.compile(r'[0-9]+_[a-zA-Z]{4}_[a-zA-Z]{4}')})

    for restaurante in restaurantes: 
        card = {}

        if restaurante.find('a', {'class': 'Lwqic Cj b'}) is not None:
            card['nome'] = restaurante.find('a', {'class': 'Lwqic Cj b'}).text
        else:
            card['nome'] = 'None'
        if restaurante.find('a', {'class': 'Lwqic Cj b'}).get('href') is not None:
            card['link'] = 'https://www.tripadvisor.com.br' + restaurante.find('a', {'class': 'Lwqic Cj b'}).get('href')
        else:
            card['link'] = 'None'

        cards.append(card)

    proxima_pagina(browser, 'a.nav:nth-child(2)')

    if fim_da_coleta == 1:
        browser.quit()
        print('Coleta finalizada!')
        break
    else:
        continue

C:\Users\guilh\AppData\Local\Temp\ipykernel_9200\709260450.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(options=options, executable_path="C:\Program Files\Mozilla Firefox\geckodriver.exe")


Erro TimeoutException: coleta pode ter sido finalizada. Verificar!
Coleta finalizada!


In [9]:
# Criando um DataFrame com os resultados e salvando em um arquivo csv:
df_restaurantes_fase1 = pd.DataFrame(cards)
df_restaurantes_fase1.to_csv('df_restaurantes_fase1.csv')